# Computational Theory 

## Problem 1 — Binary Words and Operations

Implement the SHA-256 bitwise functions: `Parity`, `Ch`, `Maj`, `Sigma0`, `Sigma1`, `sigma0`, `sigma1`.

Below we provide:

- Clear docstrings for each function.
- Explanations of the logical behaviour (algorithmic steps)
- Tests comparing small examples and showing values in hex.


### Background Reading

While researching how cryptographic hash functions work, I came across an article called [*Cryptography Hash Method MD2 (Message Digest 2) – Step-by-Step Explanation Made Easy with Python*](https://nickthecrypt.medium.com/cryptography-hash-method-md2-message-digest-2-step-by-step-explanation-made-easy-with-python-10faa2e35e85) by Nick The Crypt. Although that article focuses on the older MD2 algorithm, it helped me understand the overall logic behind hash functions how they mix and transform bits to make outputs unpredictable. MD2 and SHA-256 are quite different in their design (MD2 uses substitution tables, while SHA-256 relies on logical operations and rotations), but the core idea of achieving diffusion and non-reversibility is the same. Reading it gave me a better appreciation for why SHA-256 uses helper functions like `Ch`, `Maj`, and the Σ/σ functions I implemented here.


## Logical Explanation – Bitwise Operations and Helper Functions

The functions below form the mathematical backbone of cryptographic hash algorithms like SHA-256.  
They rely on **bitwise operations** that manipulate the binary representation of 32-bit integers to create diffusion (mixing of input bits).

1. **_rotr(x, n):**  
   This function performs a *bitwise rotation* of a 32-bit integer `x` to the right by `n` positions.  
   Unlike a normal shift, the bits that “fall off” on the right side are wrapped around to the left side.  
   This operation preserves all bits while changing their order an important property for reversible bit mixing.

2. **Parity(x, y, z):**  
   This computes the bitwise XOR (exclusive OR) of three values.  
   Each bit in the result is 1 if an odd number of corresponding bits among `x`, `y`, and `z` are 1.  
   It’s a simple way to measure “odd parity” among bits and is used to combine data unpredictably.

3. **Ch(x, y, z):**  
   Also called the *choose* function.  
   It picks bits from `y` or `z` depending on whether the corresponding bit in `x` is 1 or 0.  
   This is a conditional operation at the bit level similar to `if (x) choose y else choose z`.

4. **Maj(x, y, z):**  
   Also known as the *majority* function.  
   For each bit position, it outputs 1 if at least two of the three inputs have that bit set.  
   This simulates a logical majority vote for every bit position.

In [1]:

import numpy as np # Import NumPy for 32-bit unsigned integer support

# Helper for rotating 32-bit integers
def _rotr(x: np.uint32, n: int) -> np.uint32:
    """Rotate-right (32-bit) helper."""
     # Shift bits right by n and wrap shifted bits from the right end to the left
    return np.uint32((x >> n) | (x << (32 - n)))

# Bitwise operations used in SHA-like algorithms
def Parity(x: np.uint32, y: np.uint32, z: np.uint32) -> np.uint32:
    """Parity(x,y,z) = x XOR y XOR z."""
     # XOR returns 1 only when an odd number of inputs have 1 in that bit position
    return np.uint32(x ^ y ^ z)

def Ch(x: np.uint32, y: np.uint32, z: np.uint32) -> np.uint32:
    """Ch(x,y,z) = (x AND y) XOR ((NOT x) AND z)."""
    # For each bit in x: if bit is 1 → choose y; if bit is 0 → choose z
    return np.uint32((x & y) ^ (~x & z))

def Maj(x: np.uint32, y: np.uint32, z: np.uint32) -> np.uint32:
    """Maj(x,y,z) = (x AND y) XOR (x AND z) XOR (y AND z)."""
     # Returns 1 if the majority (two or more) of input bits are 1
    return np.uint32((x & y) ^ (x & z) ^ (y & z))


## Logical Explanation – Sigma and sigma Functions

The Sigma functions introduce **nonlinearity** and **diffusion** into the hashing process.  
They combine **bit rotations** and **bit shifts** to ensure that small changes in the input produce large, unpredictable changes in the output a property known as the *avalanche effect*.

- **Σ0(x)** and **Σ1(x)**:  
  These are used in the *main compression function* of SHA algorithms.  
  Each function applies multiple rotate-right operations and XORs the results together.  
  This mixes bits from distant positions within the 32-bit word, increasing complexity.

- **σ0(x)** and **σ1(x):**  
  These are the *small sigma* functions, used when expanding the message schedule in SHA algorithms.  
  They use a combination of rotate-right and right-shift operations (`>>`).  
  The right shift (`SHR`) introduces zeros from the left side, losing information which helps achieve more randomness and diffusion during message expansion.


In [2]:
# Uppercase Sigma functions (used in main compression)
def Sigma0(x: np.uint32) -> np.uint32:
    """Σ0(x) = ROTR^2(x) XOR ROTR^13(x) XOR ROTR^22(x)."""
      # Combine rotated versions of x to achieve bit diffusion
    return np.uint32(_rotr(x, 2) ^ _rotr(x, 13) ^ _rotr(x, 22))

def Sigma1(x: np.uint32) -> np.uint32:
    """Σ1(x) = ROTR^6(x) XOR ROTR^11(x) XOR ROTR^25(x)."""
    # Similar to Sigma0 but uses different rotation constants
    return np.uint32(_rotr(x, 6) ^ _rotr(x, 11) ^ _rotr(x, 25))

# Lowercase sigma functions (used for message schedule)
def sigma0(x: np.uint32) -> np.uint32:
    """σ0(x) = ROTR^7(x) XOR ROTR^18(x) XOR SHR^3(x)."""
     # Use rotate-right and logical right shift to mix bits in the message schedule
    return np.uint32(_rotr(x, 7) ^ _rotr(x, 18) ^ (x >> 3))

def sigma1(x: np.uint32) -> np.uint32:
    """σ1(x) = ROTR^17(x) XOR ROTR^19(x) XOR SHR^10(x)."""
      # Another mix function for message expansion with different bit positions
    return np.uint32(_rotr(x, 17) ^ _rotr(x, 19) ^ (x >> 10))


## Testing and Verification

The cell below verifies that all previously defined functions behave as expected.  
By using fixed 32-bit hexadecimal test values for `x`, `y`, and `z`, we can visually confirm the correctness of each operation.

Each `print()` statement displays the function output in **hexadecimal format** for easy reading.  
If the outputs are consistent with the expected bitwise behaviors (e.g., XOR, AND, rotations), then the implementation is correct.

These tests act as a **sanity check** before integrating the functions into larger cryptographic processes.


In [3]:
# Quick test to make sure everything works
x = np.uint32(0x12345678)
y = np.uint32(0x9abcdef0)
z = np.uint32(0x0f1e2d3c)

# Print results of each function in hexadecimal format for clarity
print("Parity:  ", f"{Parity(x,y,z):#010x}")
print("Ch:      ", f"{Ch(x,y,z):#010x}")
print("Maj:     ", f"{Maj(x,y,z):#010x}")
print("Sigma0:  ", f"{Sigma0(x):#010x}")
print("Sigma1:  ", f"{Sigma1(x):#010x}")
print("sigma0:  ", f"{sigma0(x):#010x}")
print("sigma1:  ", f"{sigma1(x):#010x}")


Parity:   0x8796a5b4
Ch:       0x1f3e7f74
Maj:      0x1a3c5e78
Sigma0:   0x66146474
Sigma1:   0x3561abda
sigma0:   0xe7fce6ee
sigma1:   0xa1f78649
